## This notebook generates the results table that goes into the paper for the Dunnnhumby data


Tricks:
save the file test_table.tex and within LaTeX:
```
\begin{table*}[h]
\centering
\resizebox{1.1\textwidth}{!}{
\input{tables/test_table.tex}
}
\caption{Explanation} 
\label{tab:ssss}
\end{table*}
```

In [1]:
import numpy as np
import uuid
import pandas as pd
import os
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
import fcn_helpers as fhelp
import glob

dataFolder = os.path.expanduser('~/Google Drive/order/Machine Learning Part/data/Dunnhumby')

mapper_family_to_category = fhelp.dunnhumby_get_map_dept_to_cat()

In [2]:
prefix = 'dunnhumby_results'
baseFolder = '~/Google Drive/order/Machine Learning Part/Preparing the 3rd paper/examples for the paper/tex'
tex_file_name = os.path.join(os.path.expanduser(baseFolder), prefix + '_table.tex')

In [3]:
def writeTextFile(thisStr, thisFile):
    with open(thisFile, 'w') as f:
        f.write(thisStr)

In [4]:
def get_results_from_analysis(path_to_file
    ,save_all_cannibals = True
    ,min_diff_in_units_from_reg_to_promo = 10
    ,min_promo_days=3
    ,min_regular_days=6
    ,min_ratio_change = 0.5
    ,do_exclude_promos_SKU_B = True
    ,be_verbose=False
    ,sales_threshold = 1/3
    ,min_avg_sales = 10
    ,period_in_days = 7
    ,do_decomposition = False
    ,cannibalisation_threshold = 1/3
    ,min_snap_days = 10):
    '''
        * category
        * department
        * store
        * number skus > total_number_skus
        * number skus that have got promotions and are worth investigating > num_skus_promotions
        * number skus that could be cannibals > num_sku_potential_cannibals
        * number of combinations analysed > num_combinations_analysed
        * number of cannibals > number_cannibals
        * number of victims > number_victims
        * number of cannibalisation episodes > num_cannibalisation_episodes
        * percentage of cannibalisation > avg_can_percentage, std_can_percentage
    '''
    causal_impact_results = pd.read_pickle(path_to_file)
    
    dict_results = {}

    if causal_impact_results.empty:
      return
    
    current_store_name = path_to_file.split('/')[-1].split('.')[0]

    sku_A = f'sales-{causal_impact_results.cannibal[0]}-{current_store_name}'
    # Get the taxonomy
    _, dept_id, sku_id, store_name = fhelp.get_taxonomy_from_sku_name_CFAV(sku_A)
    category_id = mapper_family_to_category.get(dept_id, '')

    # read the stores data
    foldername = os.path.join(dataFolder, category_id, dept_id, 'store_sales')
    filename = os.path.join(foldername, f'{store_name}.pickle')
    df_store = fhelp.readPickleFile(filename)

    sales_vars = [iVar for iVar in df_store.columns if 'sales-' in iVar]
    promotion_vars = [iVar for iVar in df_store.columns if 'promotion_flag-' in iVar]

    total_number_skus = len(sales_vars)


    '''
      number skus that could be cannibals
    '''

    folderPromoStats = os.path.join(os.path.expanduser(dataFolder), 'sku_promo_slot_analysis', category_id, dept_id)
    filename = os.path.join(folderPromoStats, f'{store_name}.pickle')


    if os.path.exists(filename):
      df_snap_stats = fhelp.readPickleFile(filename)

      num_skus_promotions = len(df_snap_stats.index.tolist())

      # A bit of work on the uplifters
      # Snap sales greater than the sales + threshold
      idx_A = df_snap_stats['mu_delta']>=(1+sales_threshold)

      # Comparison only valid if they are not zero sales
      idx_B = (df_snap_stats['avg_promo_sales']>min_avg_sales) & (df_snap_stats['avg_regular_sales']>min_avg_sales)
      df_snap_stats['uplift_in_median'] = (idx_A & idx_B)

      # Get two groups: potential cannibals (and haloers) and victims
      df_snap_uplifters = df_snap_stats[df_snap_stats['uplift_in_median']].copy()
      sku_potential_cannibals = df_snap_uplifters.index.tolist()
      num_sku_potential_cannibals = len(sku_potential_cannibals)
    else:
      return
    


    '''
      number of combinations analysed
      number cannibals * potential_victims * num_promotional_slots
      potential_victims = total_number_skus-1

      num_promotional_slots can be approximated as df_snap_stats.num_promo_slots.mean()
    '''

    num_combinations_analysed = \
      round(num_sku_potential_cannibals*(total_number_skus-1)*df_snap_stats.num_promo_slots.mean())

    '''
      Final numbers
    '''
    current_cannibals = causal_impact_results.cannibal.unique().tolist()
    current_victims = causal_impact_results.victim.unique().tolist()
    number_cannibals = len(current_cannibals)
    number_victims = len(current_victims)

    num_cannibalisation_episodes = causal_impact_results.shape[0]

    percentage_cannibalisation = \
      100*causal_impact_results.sku_B_avg_sales_during_promo_sku_A/causal_impact_results.sku_B_regular_avg_sales

    avg_can_percentage = percentage_cannibalisation.mean()
    std_can_percentage = percentage_cannibalisation.std()    
    
    '''
      average daily losses (units)
    '''
    avg_abs_effect = causal_impact_results.avg_abs_effect.mean()
    std_abs_effect = causal_impact_results.avg_abs_effect.std()


    '''
      average cumulative lost due to cannibalisation (units)
    '''
    avg_cum_abs_effect = causal_impact_results.cum_abs_effect.mean()
    total_cum_abs_effect = causal_impact_results.cum_abs_effect.sum()

    '''
    Average probability of Causal Effect
    '''
    avg_prob_causal_effect = causal_impact_results.prob_causal_effect.mean()

    dict_results = {'category': category_id,
    'department': dept_id,
    'store': store_name,
    'total_number_skus': total_number_skus,
    'num_skus_promotions': num_skus_promotions,
    'num_sku_potential_cannibals': num_sku_potential_cannibals,
    'num_combinations_analysed': num_combinations_analysed,
    'number_cannibals': number_cannibals,
    'number_victims': number_victims,
    'num_cannibalisation_episodes': num_cannibalisation_episodes,
    'avg_can_percentage': avg_can_percentage,
    'std_can_percentage': std_can_percentage,
    'avg_abs_effect': avg_abs_effect,
    'std_abs_effect': std_abs_effect,
    'avg_cum_abs_effect': avg_cum_abs_effect,
    'total_cum_abs_effect': total_cum_abs_effect, 
    'avg_prob_causal_effect': avg_prob_causal_effect}
    
    return dict_results

In [5]:
glob_pattern = os.path.join(os.path.expanduser(dataFolder), 'CausalImpactResults', '*', '*', '*.pickle')
store_level_files = glob.glob(glob_pattern)

In [6]:
all_results = []
for this_store in store_level_files:
    current_results = get_results_from_analysis(this_store)
    if current_results:
        all_results.append(current_results)

In [7]:
df_results = pd.DataFrame(all_results)
df_results.sort_values(by=['category', 'department', 'store'], inplace=True)
fcn_clean_str = lambda s: s.replace('.', ' ')
df_results['category'] = df_results['category'].apply(fcn_clean_str)
df_results['department'] = df_results['department'].apply(fcn_clean_str)
df_results.fillna(0, inplace=True)
df_results.head()

,category,department,store,total_number_skus,num_skus_promotions,num_sku_potential_cannibals,num_combinations_analysed,number_cannibals,number_victims,num_cannibalisation_episodes,avg_can_percentage,std_can_percentage,avg_abs_effect,std_abs_effect,avg_cum_abs_effect,total_cum_abs_effect,avg_prob_causal_effect
8,BAG SNACKS,PRETZELS,OH_21237_MAINSTREAM_1,15,13,2,73,1,1,1,40.840841,0.000000,-14.213226,0.000000,-42.639679,-42.639679,85.114885
7,BAG SNACKS,PRETZELS,OH_2277_UPSCALE_1,15,13,3,136,2,1,4,37.511515,10.866778,-42.371359,16.147265,-90.825423,-363.301693,92.657343
6,BAG SNACKS,PRETZELS,OH_24991_UPSCALE_1,15,15,5,210,1,1,1,47.922849,0.000000,-10.223116,0.000000,-20.446232,-20.446232,93.506494
9,BAG SNACKS,PRETZELS,OH_25027_MAINSTREAM_1,15,14,3,165,1,1,1,25.743349,0.000000,-37.163387,0.000000,-111.490161,-111.490161,72.527473
11,BAG SNACKS,PRETZELS,OH_6179_UPSCALE_1,15,15,4,119,2,3,3,41.004260,9.964356,-26.276305,14.247521,-62.094994,-186.284983,95.171495


In [8]:
total_cannibals = df_results.number_cannibals.sum()
total_episodes  = df_results.num_cannibalisation_episodes.sum()
total_cum_abs_effect = df_results.total_cum_abs_effect.sum()
avg_percentage = df_results.avg_can_percentage.mean()

total_cannibals, total_episodes, total_cum_abs_effect, avg_percentage

(26, 49, -2855.6264088228218, 37.874195368672275)

Adapt the following to show the table

['category',
 'department',
 'store',
 'total_number_skus',
 'num_skus_promotions',
 'num_sku_potential_cannibals',
 'num_combinations_analysed',
 'number_cannibals',
 'number_victims',
 'num_cannibalisation_episodes',
 'avg_can_percentage',
 'std_can_percentage',
 'avg_abs_effect',
 'std_abs_effect',
 'avg_cum_abs_effect',
 'total_cum_abs_effect',
 'avg_prob_causal_effect']

for iCol in df_results.columns.tolist():
    print(f'\'{iCol}\':\'{iCol}\',')

In [9]:
df_results['percentage'] = df_results[['avg_can_percentage', 'std_can_percentage']].apply(lambda st: f'${st[0]:3.1f} PLUSMINUS {st[1]:3.1f}$', axis=1)
df_results['average_daily_losses'] = df_results[['avg_abs_effect', 'std_abs_effect']].apply(lambda st: f'${st[0]:3.1f} PLUSMINUS {st[1]:3.1f}$', axis=1)
df_results['cat_dep'] = df_results[['category', 'department']].apply(lambda st: f'{st[0]}-{st[1]}', axis=1)
df_results['store_num'] = df_results['store'].apply(lambda st: st.split('_')[1])

df_results['sku_promo'] = df_results[['total_number_skus', 'num_skus_promotions']].apply(lambda st: f'{st[0]:d}/{st[1]:d}', axis=1)

df_results.head()

,category,department,store,total_number_skus,num_skus_promotions,num_sku_potential_cannibals,num_combinations_analysed,number_cannibals,number_victims,num_cannibalisation_episodes,...,avg_abs_effect,std_abs_effect,avg_cum_abs_effect,total_cum_abs_effect,avg_prob_causal_effect,percentage,average_daily_losses,cat_dep,store_num,sku_promo
8,BAG SNACKS,PRETZELS,OH_21237_MAINSTREAM_1,15,13,2,73,1,1,1,...,-14.213226,0.000000,-42.639679,-42.639679,85.114885,$40.8 PLUSMINUS 0.0$,$-14.2 PLUSMINUS 0.0$,BAG SNACKS-PRETZELS,21237,15/13
7,BAG SNACKS,PRETZELS,OH_2277_UPSCALE_1,15,13,3,136,2,1,4,...,-42.371359,16.147265,-90.825423,-363.301693,92.657343,$37.5 PLUSMINUS 10.9$,$-42.4 PLUSMINUS 16.1$,BAG SNACKS-PRETZELS,2277,15/13
6,BAG SNACKS,PRETZELS,OH_24991_UPSCALE_1,15,15,5,210,1,1,1,...,-10.223116,0.000000,-20.446232,-20.446232,93.506494,$47.9 PLUSMINUS 0.0$,$-10.2 PLUSMINUS 0.0$,BAG SNACKS-PRETZELS,24991,15/15
9,BAG SNACKS,PRETZELS,OH_25027_MAINSTREAM_1,15,14,3,165,1,1,1,...,-37.163387,0.000000,-111.490161,-111.490161,72.527473,$25.7 PLUSMINUS 0.0$,$-37.2 PLUSMINUS 0.0$,BAG SNACKS-PRETZELS,25027,15/14
11,BAG SNACKS,PRETZELS,OH_6179_UPSCALE_1,15,15,4,119,2,3,3,...,-26.276305,14.247521,-62.094994,-186.284983,95.171495,$41.0 PLUSMINUS 10.0$,$-26.3 PLUSMINUS 14.2$,BAG SNACKS-PRETZELS,6179,15/15


In [10]:
aggregations  = {'total_number_skus': 'mean', 'num_skus_promotions': 'mean'}
df_results_agg = df_results.groupby(['category', 'department', ], as_index=False).agg(aggregations)
df_results_agg.sort_values(by=['category', 'department'], inplace=True)
df_results_agg['cat_dep'] = df_results_agg[['category', 'department']].apply(lambda st: f'{st[0]}-{st[1]}', axis=1)
vars_to_save = ['cat_dep', 'total_number_skus', 'num_skus_promotions']
str_latex = df_results_agg[vars_to_save].to_latex(index=False, float_format='{:3.2f}'.format)
print(str_latex)

\begin{tabular}{lrr}
\toprule
                       cat\_dep &  total\_number\_skus &  num\_skus\_promotions \\
\midrule
           BAG SNACKS-PRETZELS &                 15 &                14.14 \\
 COLD CEREAL-ALL FAMILY CEREAL &                  7 &                 6.00 \\
       COLD CEREAL-KIDS CEREAL &                  5 &                 4.20 \\
    FROZEN PIZZA-PIZZA PREMIUM &                 15 &                11.60 \\
\bottomrule
\end{tabular}



In [11]:
fhelp.to_random_excel_file(df_results)

In [12]:
d = {'total_number_skus':'skus',     
'num_skus_promotions':'promotions',
'num_sku_potential_cannibals':'potential cannibals',
'num_combinations_analysed': 'combinations analysed',
'number_cannibals':'cannibals',
'number_victims':'victims',
'num_cannibalisation_episodes':'episodes',
     
'avg_can_percentage':'avg_can_percentage',
'std_can_percentage':'std_can_percentage',
     
'avg_abs_effect':'avg_abs_effect',
'std_abs_effect':'std_abs_effect',
     
'total_cum_abs_effect':'Cum abs effect',
'avg_prob_causal_effect':'P_{CE}'
}

In [13]:
df_results.rename(columns=d, inplace=True)
df_results.head()

,category,department,store,skus,promotions,potential cannibals,combinations analysed,cannibals,victims,episodes,...,avg_abs_effect,std_abs_effect,avg_cum_abs_effect,Cum abs effect,P_{CE},percentage,average_daily_losses,cat_dep,store_num,sku_promo
8,BAG SNACKS,PRETZELS,OH_21237_MAINSTREAM_1,15,13,2,73,1,1,1,...,-14.213226,0.000000,-42.639679,-42.639679,85.114885,$40.8 PLUSMINUS 0.0$,$-14.2 PLUSMINUS 0.0$,BAG SNACKS-PRETZELS,21237,15/13
7,BAG SNACKS,PRETZELS,OH_2277_UPSCALE_1,15,13,3,136,2,1,4,...,-42.371359,16.147265,-90.825423,-363.301693,92.657343,$37.5 PLUSMINUS 10.9$,$-42.4 PLUSMINUS 16.1$,BAG SNACKS-PRETZELS,2277,15/13
6,BAG SNACKS,PRETZELS,OH_24991_UPSCALE_1,15,15,5,210,1,1,1,...,-10.223116,0.000000,-20.446232,-20.446232,93.506494,$47.9 PLUSMINUS 0.0$,$-10.2 PLUSMINUS 0.0$,BAG SNACKS-PRETZELS,24991,15/15
9,BAG SNACKS,PRETZELS,OH_25027_MAINSTREAM_1,15,14,3,165,1,1,1,...,-37.163387,0.000000,-111.490161,-111.490161,72.527473,$25.7 PLUSMINUS 0.0$,$-37.2 PLUSMINUS 0.0$,BAG SNACKS-PRETZELS,25027,15/14
11,BAG SNACKS,PRETZELS,OH_6179_UPSCALE_1,15,15,4,119,2,3,3,...,-26.276305,14.247521,-62.094994,-186.284983,95.171495,$41.0 PLUSMINUS 10.0$,$-26.3 PLUSMINUS 14.2$,BAG SNACKS-PRETZELS,6179,15/15


In [14]:
print(','.join(sorted(df_results['store_num'].unique().tolist())))

19265,21227,21237,2277,2281,24991,25021,25027,4259,613,6179,8041,9825


In [15]:
vars_to_save = ['cat_dep',
 'store_num',
 'sku_promo',
 'combinations analysed',
 'cannibals',
 'victims',
 'episodes',
 'P_{CE}',
 'percentage',
 'avg_cum_abs_effect',
 'average_daily_losses']
df_latex = df_results[vars_to_save].copy()
df_latex.head()

,cat_dep,store_num,sku_promo,combinations analysed,cannibals,victims,episodes,P_{CE},percentage,avg_cum_abs_effect,average_daily_losses
8,BAG SNACKS-PRETZELS,21237,15/13,73,1,1,1,85.114885,$40.8 PLUSMINUS 0.0$,-42.639679,$-14.2 PLUSMINUS 0.0$
7,BAG SNACKS-PRETZELS,2277,15/13,136,2,1,4,92.657343,$37.5 PLUSMINUS 10.9$,-90.825423,$-42.4 PLUSMINUS 16.1$
6,BAG SNACKS-PRETZELS,24991,15/15,210,1,1,1,93.506494,$47.9 PLUSMINUS 0.0$,-20.446232,$-10.2 PLUSMINUS 0.0$
9,BAG SNACKS-PRETZELS,25027,15/14,165,1,1,1,72.527473,$25.7 PLUSMINUS 0.0$,-111.490161,$-37.2 PLUSMINUS 0.0$
11,BAG SNACKS-PRETZELS,6179,15/15,119,2,3,3,95.171495,$41.0 PLUSMINUS 10.0$,-62.094994,$-26.3 PLUSMINUS 14.2$


In [16]:
str_latex = df_latex.to_latex(index=False, float_format='{:3.2f}'.format)

str_latex= str_latex.replace('PLUSMINUS', '\pm')
#str_latex= str_latex.replace('\_', ' ')
str_latex= str_latex.replace('\$', '$')
#print(str_latex)

In [17]:
writeTextFile(str_latex, tex_file_name)
print(f'File written to {tex_file_name}')

File written to /Users/carlos.aguilar/Google Drive/order/Machine Learning Part/Preparing the 3rd paper/examples for the paper/tex/dunnhumby_results_table.tex


In [18]:
print(str_latex)

\begin{tabular}{lllrrrrrlrl}
\toprule
                       cat\_dep & store\_num & sku\_promo &  combinations analysed &  cannibals &  victims &  episodes &  P\_\{CE\} &             percentage &  avg\_cum\_abs\_effect &    average\_daily\_losses \\
\midrule
           BAG SNACKS-PRETZELS &     21237 &     15/13 &                     73 &          1 &        1 &         1 &   85.11 &   $40.8 \pm 0.0$ &              -42.64 &   $-14.2 \pm 0.0$ \\
           BAG SNACKS-PRETZELS &      2277 &     15/13 &                    136 &          2 &        1 &         4 &   92.66 &  $37.5 \pm 10.9$ &              -90.83 &  $-42.4 \pm 16.1$ \\
           BAG SNACKS-PRETZELS &     24991 &     15/15 &                    210 &          1 &        1 &         1 &   93.51 &   $47.9 \pm 0.0$ &              -20.45 &   $-10.2 \pm 0.0$ \\
           BAG SNACKS-PRETZELS &     25027 &     15/14 &                    165 &          1 &        1 &         1 &   72.53 &   $25.7 \pm 0.0$ &             -111.49 &  